In [1]:
!pip install -q streamlit
!pip install -q -U google-generativeai
!pip install --upgrade openai
!pip install --upgrade streamlit-mic-recorder
!pip install --upgrade streamlit-realtime-audio-recorder
!pip install pyngrok
!pip install av
!pip install streamlit-TTS
!pip install python-dotenv
!pip install pyttsx3
!pip install streamlit-image-viewer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.4/446.4 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s et

In [26]:
%%writefile app.py
import streamlit as st
import google.generativeai as genai
import openai
import os
import io
from os.path import isfile
from PIL import Image
import speech_recognition as sr
import base64
from pydub import AudioSegment
from io import BytesIO
from streamlit_mic_recorder import mic_recorder, speech_to_text
from streamlit_realtime_audio_recorder import audio_recorder
from pathlib import Path
from datetime import datetime
from streamlit_TTS import auto_play, text_to_speech, text_to_audio
import pyttsx3
from google.colab import userdata
# from streamlit_webrtc import webrtc_streamer, AutoProcessorBase
import av

# Configure Google Generative AI (replace with Streamlit Secrets for production)
genai.configure(api_key="***") # Replace with your actual Gemini API key

# Configure OpenAI API (replace with Streamlit Secrets for production)
client = openai.OpenAI(api_key = "***")

st.set_page_config(layout="centered") # Optional: Set layout for better appearance

st.title("Recruiting AInterviewee") # Persona Creating LLM Demo

# --- Initialize Session State Variables ---
if 'persona_generated' not in st.session_state:
    st.session_state.persona_generated = False
if 'interview_started' not in st.session_state:
    st.session_state.interview_started = False
if 'interview_ended' not in st.session_state:
    st.session_state.interview_ended = False
if 'messages' not in st.session_state:
    st.session_state.messages = [{'role': 'system', 'content': '안녕.'}]
if 'interviewer_question_log' not in st.session_state:
    st.session_state.interviewer_question_log = ""
if 'interviewee_reply_log' not in st.session_state:
    st.session_state.interviewee_reply_log = ""
if 'persona' not in st.session_state:
    st.session_state.persona = ""
if 'name' not in st.session_state:
    st.session_state.name = ""
if 'audio_processed' not in st.session_state:
    st.session_state.audio_processed = False
if 'audio_input_key_counter' not in st.session_state:
    st.session_state.audio_input_key_counter = 0
if 'scenario' not in st.session_state:
    st.session_state.scenario = ""
if 'value_scenario' not in st.session_state:
    st.session_state.value_scenario = ""
if 'interview_topic' not in st.session_state:
    st.session_state.interview_topic = ""
if 'user_response_voluntarity' not in st.session_state:
    st.session_state.user_response_voluntarity = "예"
if 'user_age' not in st.session_state:
    st.session_state.user_age = ""
if 'user_gender' not in st.session_state:
    st.session_state.user_gender = ""
if 'user_nation' not in st.session_state:
    st.session_state.user_nation = ""
# Add a state for the current screen
if 'current_screen' not in st.session_state:
    st.session_state.current_screen = 'persona_input' # Default screen

def navigate_to(screen_name):
    """Changes the current screen and reruns the app."""
    st.session_state.current_screen = screen_name
    st.rerun()

# --- Screen Functions ---

def persona_input_screen():
    persona_input_screen_title = '1. Persona Information Input'
    st.markdown(persona_input_screen_title, unsafe_allow_html=True)
    # st.header("1. Persona Information Input", width=1000)
    interview_topic = st.text_input(label = "Interview Topic", value=st.session_state.interview_topic)
    user_response_voluntarity = st.radio("Is the Virtual Interviewee Cooperative?", ('Yes', 'No'), index=0 if st.session_state.user_response_voluntarity == 'Yes' else 1)
    user_response_length = "Long" if user_response_voluntarity == 'Yes' else "Short"
    user_age = st.text_input(label = "Age", value=st.session_state.user_age)
    # user_gender = st.text_input("Gender", value=st.session_state.user_gender)
    user_gender = st.radio("Gender", ('Male', 'Female', 'Other'), index=0 if st.session_state.user_gender == 'Male' else 1 if st.session_state.user_gender == 'Female' else 2)
    user_nation = st.text_input(label = "Nationality", value=st.session_state.user_nation)

    # Store inputs in session state to persist across reruns
    st.session_state.interview_topic = interview_topic
    st.session_state.user_response_voluntarity = user_response_voluntarity
    st.session_state.user_age = user_age
    st.session_state.user_gender = user_gender
    st.session_state.user_nation = user_nation

    first_step_input = f"Topic: {interview_topic} / Response Length: {user_response_length} / Age: {user_age} / Gender: {user_gender} / Nationality: {user_nation}"

    if st.button("Persona Generate & Interview Start"):
        # Validate inputs
        if not all([interview_topic, user_age, user_gender, user_nation]):
            st.error("Please enter all of the persona informations.")
            st.session_state.persona_generated = False
        else:
            st.session_state.persona_generated = True
            st.session_state.interview_started = False # Reset if regenerating persona
            st.session_state.interview_ended = False # Reset if regenerating persona
            st.session_state.interviewer_question_log = ""
            st.session_state.interviewee_reply_log = ""
            st.session_state.messages = [{'role': 'system', 'content': '안녕.'}] # Reset chat history

            with st.spinner("Generating Persona..."):
                model = genai.GenerativeModel('gemini-2.5-flash')
                try:
                    # Generate Persona
                    persona_response = model.generate_content([first_step_input, " Please make a virtual person persona based on the information given prior. Please provide topic of the interview, cooperativeness of the interviewee, age, gender and nationality in the output. For the output, I also need the name, education level, the problem this person is facing, the lifestyle of the person, the person's interests, his family, the objective of this interview, and religion."])
                    st.session_state.persona = persona_response.text

                    # Generate Name
                    name_response = model.generate_content([st.session_state.persona, "Tell me ONLY this person's name."])
                    st.session_state.name = name_response.text.strip()

                    # Generate Scenario
                    scenario_response = model.generate_content([st.session_state.persona, "Make a short scenario based on the persona information."])
                    st.session_state.scenario = scenario_response.text

                    # Generate Value Scenario
                    definition_of_value_scenario = 'Value scenario: ‘‘‘An extension of scenario-based design which can support envisioning the systemic effects of new technologies. There are elements of value scenarios. Value scenarios draw upon five key elements to develop provocative sketches of the future: stakeholders, pervasiveness, time, systemic effects, and value implications. Stakeholders: Following Value Sensitive Design, value scenarios help designers envision a range of effects of a pervasive technology, both on those who are in direct contact with a technology (direct stakeholders), and on those who might not be direct users, but whose lives are affected by various interactions around the technology (indirect stakeholders). Pervasiveness: A value scenario presents a vision in which a technology has become widespread, spanning various geographic regions, cultures, social classes, and other contexts (e.g. school, work, home, car). Time: Rather than focus on short-term effects, value scenarios take into consideration what the world might look like five, ten, or twenty years after a technology has been deployed. Systemic Effects: Value scenarios explore the multi-dimensional interactions among technology, psychology, society, culture, and the environment as use of the technology becomes pervasive over a period of years. Value Implications: Finally, drawing on Value Sensitive Design and aspects of design noir, value scenarios help envision not only positive effects of technology, but also its darker consequences. We suggest that a careful consideration of a diverse range of influences, including the negative, should be a key component of the design process.‘‘‘'
                    value_scenario_response = model.generate_content([definition_of_value_scenario, "This is the definition of a value scenario.", st.session_state.persona, 'This is the virtual interviewee persona.', st.session_state.scenario, 'This is a short scenario about the person the persona is explaining about. Based on the persona and this short scenario, write me a value scenario.'])
                    st.session_state.value_scenario = value_scenario_response.text

                    # Initialize chat history with persona context for OpenAI
                    st.session_state.messages.append({"role": "system", "content": st.session_state.persona + f": This is {st.session_state.name}'s persona."})
                    counsel_create = client.chat.completions.create(model = 'gpt-3.5-turbo', messages = st.session_state.messages)
                    reply = counsel_create.choices[0].message.content
                    st.session_state.messages.append( {"role": "system", "content": reply})

                    st.session_state.messages.append({'role': 'system', 'content': st.session_state.scenario + f" : This is the scenario {st.session_state.name} is facing."})
                    counsel_create = client.chat.completions.create(model = 'gpt-3.5-turbo', messages = st.session_state.messages)
                    reply = counsel_create.choices[0].message.content
                    st.session_state.messages.append( {"role": "system", "content": reply})

                    st.session_state.messages.append({'role': 'system', 'content': st.session_state.value_scenario + f" : This is the value scenario {st.session_state.name} is facing."})
                    counsel_create = client.chat.completions.create(model = 'gpt-3.5-turbo', messages = st.session_state.messages)
                    reply = counsel_create.choices[0].message.content
                    st.session_state.messages.append( {"role": "system", "content": reply})

                    st.session_state.messages.append({'role': 'system', 'content': f"From now on, you are going to be {st.session_state.name}'s persona. I am the interviewer, and you are going to undergo a pilot interview as the interviewee. Got it?"})
                    counsel_create = client.chat.completions.create(model = 'gpt-3.5-turbo', messages = st.session_state.messages)
                    reply = counsel_create.choices[0].message.content
                    st.session_state.messages.append( {"role": "system", "content": reply})

                    st.session_state.interview_started = True # Mark interview as ready to start
                    navigate_to('persona_display') # Navigate to persona display after generation
                except Exception as e:
                    st.error(f"An error occured while generating the persona: {e}")
                    st.session_state.persona_generated = False
                del model

def persona_display_screen():
    st.header("2. Information on the Generated Persona")
    st.write(st.session_state.persona)
    # st.subheader("시나리오:") # Uncomment to display scenario
    # st.write(st.session_state.scenario)
    # st.subheader("Value Scenario:") # Uncomment to display value scenario
    # st.write(st.session_state.value_scenario)

    if st.button(f"Begin Interview With {st.session_state.name}"):
        navigate_to('interview')

    if st.button("Generate New Persona"):
        # Reset session state for a new persona
        st.session_state.persona_generated = False
        st.session_state.interview_started = False
        st.session_state.interview_ended = False
        st.session_state.messages = [{'role': 'system', 'content': 'Hi.'}]
        st.session_state.interviewer_question_log = ""
        st.session_state.interviewee_reply_log = ""
        st.session_state.persona = ""
        st.session_state.name = ""
        st.session_state.scenario = ""
        st.session_state.value_scenario = ""
        st.session_state.interview_topic = ""
        st.session_state.user_response_voluntarity = "Yes"
        st.session_state.user_age = ""
        st.session_state.user_gender = ""
        st.session_state.user_nation = ""
        navigate_to('persona_input')

def transcribe_audio(audio_filepath):
    r = sr.Recognizer()
    with sr.AudioFile(audio_filepath) as source:
        audio = r.record(source)
    try:
        text = r.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return "Could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service: {e}"

# def TTS(response):
#   response = client.audio.speech.create(model="gpt-4o-mini-tts", voice = "onyx", input=response)
#   filename = "output.mp3"
#   response.stream_to_file(filename)
#   with open(filename, "rb") as f:
#     data = f.read()
#   b64 = base64.b64encode(data).decode()
#   md = f"""
#
#   """
#   st.markdown(md, unsafe_allow_html=True)
#   os.remove(filename)
#   # st.audio(filename, format = "audio/mp3", autoplay=True)

def interview_screen():
    interview_screen_title = f'3. Interview with {st.session_state.name}'
    st.markdown(interview_screen_title, unsafe_allow_html=True)
    # st.header(f"3. Interview with {st.session_state.name}", width=1000)

    picture_name = "man_image.png"
    # with st.spinner("Generating Profile Picture..."):
    #   try:
    #     model = genai.GenerativeModel('gemini-2.5-flash')
    #     picture_response = model.generate_content([st.session_state.persona, "Can you determine whether the person explained in this persona is either a foreigner / grandma / or a korean man? ONLY GIVE THE RESULT AS 'FOREIGNER' OR 'GRANDMA' OR 'MAN'?"])
    #     if not picture_response.parts:
    #       st.error("The model blocked the response due to safety filters.")
    #       st.stop()
    #     if picture_response.text == 'FOREIGNER': picture_name = "foreigner_image.png"
    #     elif picture_response.text == 'GRANDMA': picture_name = 'grandma_image.png'
    #     else: picture_name = "man_image.png"
    #     del model
    #   except Exception as e:
    #     st.error(f"An error occured while generating the persona: {e}")
    #     st.session_state.persona_generated = False

    if int(st.session_state.user_age) > 65 and st.session_state.user_gender == 'Female': picture_name = "grandma_image.png"
    elif int(st.session_state.user_nation) != "Korea": picture_name = "foreigner_image.png"

    st.image(picture_name, caption = 'Profile picture')

    if st.button('End Interview'):
      navigate_to('evaluation')

    # Interviewer input
    if not st.session_state.interview_ended:
        if not st.session_state.audio_processed:
            # Render the audio recorder only when we expect audio input
            st.write("Waiting for Interviewee Question Recording...")
            audio_value = audio_recorder(interval=50, threshold=-60, silenceTimeout=2000)

            if audio_value:
                if audio_value.get('status') == 'stopped':
                    audio_data = audio_value.get('audioData')
                    if audio_data:
                        audio_bytes = base64.b64decode(audio_data)

                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                        filename = f"recorded_audio_{timestamp}.webm"

                        try:
                            with open(filename, "wb") as f:
                                f.write(audio_bytes)

                            if isfile(filename):
                                st.success(f"The Audio File is Saved: {filename}")

                            audio_file = io.BytesIO(audio_bytes)
                            st.audio(audio_file, format="audio/webm")

                            # Store filename in session state to access it in the next rerun
                            st.session_state.current_audio_filename = filename
                            st.session_state.audio_processed = True
                            st.rerun() # Rerun to process the audio
                        except Exception as e:
                            st.error(f"An error occured while saving the audio file: {e}")
                            st.session_state.audio_processed = False # Reset if error
                            st.session_state.audio_input_key_counter += 1
                            st.rerun()
                elif audio_value.get('status') == 'recording':
                    st.write("Recording...")
                elif audio_value.get('error'):
                    st.error(audio_value.get('error'))
            else:
                st.write("Getting Ready for Recording...")
        else:
            # Audio has been processed, now transcribe and generate response
            filename = st.session_state.get('current_audio_filename')
            if filename and isfile(filename):
                try:
                    st.info("Transcribing Audio to Text...")
                    with open(filename, "rb") as file:
                        transcript = client.audio.transcriptions.create(model="whisper-1", file=file)
                    text = transcript.text
                    st.write(f"Interviewer: {text}")
                    st.session_state.interviewer_question_log += text + " / "

                    if text:
                        st.session_state.messages.append({"role": "user", "content": text})
                        st.chat_message("user").write(text)
                        with st.spinner("Creating response..."):
                            try:
                                counsel_create = client.chat.completions.create(model='gpt-3.5-turbo', messages=st.session_state.messages)
                                reply = counsel_create.choices[0].message.content
                                # engine = pyttsx3.init()
                                # engine.say(reply)
                                # engine.runAndWait()
                                reply_audio = text_to_audio(reply, language='en')
                                auto_play(reply_audio)
                                speech_file_path = Path(__file__).parent / "speech.mp3"
                                with client.audio.speech.with_streaming_response.create(model="gpt-4o-mini-tts", voice="coral", input=reply, instructions="Speak in a cheerful and positive tone.") as response:
                                    response.stream_to_file(speech_file_path)
                                st.audio(str(speech_file_path), format='audio/mp3', autoplay=True)
                                st.session_state.interviewee_reply_log += reply + " / "
                                st.session_state.messages.append({"role": "assistant", "content": reply})
                                st.chat_message("assistant").write(reply)

                            except Exception as e:
                                st.error(f"An error occured while creating a reply: {e}")
                    else:
                        st.warning("The transcribed text is empty.")

                except Exception as e:
                    st.error(f"An error occured while transcribing the audio to text: {e}")
                finally:
                    # Reset for the next audio input
                    st.session_state.audio_processed = False
                    st.session_state.audio_input_key_counter += 1
                    # Clean up the temporary file after processing
                    if isfile(filename):
                        import os
                        os.remove(filename)
                    st.rerun() # Rerun to re-render audio_recorder for the next input
            else:
                st.error("There is no audio file to process. Please try again.")
                st.session_state.audio_processed = False
                st.session_state.audio_input_key_counter += 1
                st.rerun()

    for msg in st.session_state.messages:
        if msg["role"] != "system": # Don't display system messages
            st.chat_message(msg["role"]).write(msg["content"])

def evaluation_screen():
    st.header("4. Interview Evaluation")

    # Check both logs for content
    if not st.session_state.interviewer_question_log and not st.session_state.interviewee_reply_log:
        st.warning("There is little information on the interview.")
    else:
        # Calculate utterance ratio
        interviewer_utterances = [u for u in st.session_state.interviewer_question_log.split(" / ") if u.strip()]
        interviewee_utterances = [u for u in st.session_state.interviewee_reply_log.split(" / ") if u.strip()]

        interviewer_count = len(st.session_state.interviewer_question_log)
        interviewee_count = len(st.session_state.interviewee_reply_log)

        total_utterances = interviewer_count + interviewee_count
        if total_utterances > 0:
            utterance_ratio = interviewee_count / total_utterances
            st.write(f"Utterance Ratio (Interviewee Responses / Total Utterances): **{utterance_ratio:.3f}**")

            if utterance_ratio > 0.8: # A more balanced ratio
                st.success("The Utterance Ratio was **adequate!** Great Interview!")
            else:
                st.warning("The Utterance Ratio of the Interviewee was **too low.** Pay more attention to the response or ask broader questions.")
        else:
            st.write("No utterances recorded during the interview.")

        # Display logs
        st.subheader("Interview Logs")
        with st.expander("Show Interviewer Questions Log"):
            st.text_area("Interviewer Questions", st.session_state.interviewer_question_log, height=150)
        with st.expander("Show Interviewee Replies Log"):
            st.text_area("Interviewee Replies", st.session_state.interviewee_reply_log, height=150)


        # Generate interview evaluation
        evaluate_message = [{'role': 'system', 'content': 'You are an interview evaluator. Provide a concise summary of the interview, highlighting key takeaways. Then, identify what went well, what could be improved, and offer specific, actionable recommendations for improvement based on the interviewer questions and interviewee replies provided. Always maintain a professional and constructive tone.'}]
        evaluate_message.append({"role": "user", "content": f"Interviewer questions: {st.session_state.interviewer_question_log}"})
        evaluate_message.append({"role": "user", "content": f"Interviewee replies: {st.session_state.interviewee_reply_log}"})

        with st.spinner("Evaluating Interview..."):
            try:
                counsel_evaluate = client.chat.completions.create(model = 'gpt-3.5-turbo', messages = evaluate_message)
                evaluation_reply = counsel_evaluate.choices[0].message.content
                st.subheader("Interview Feedback Result:")
                st.write(evaluation_reply)
            except Exception as e:
                st.error(f"An error occured while assessing the interview: {e}")

    # Option to restart
    if st.button("Begin New Interview"):
        # Reset all relevant session state variables
        st.session_state.persona_generated = False
        st.session_state.interview_started = False
        st.session_state.interview_ended = False
        st.session_state.messages = [{'role': 'system', 'content': 'Hi.'}]
        st.session_state.interviewer_question_log = ""
        st.session_state.interviewee_reply_log = ""
        st.session_state.persona = ""
        st.session_state.name = ""
        st.session_state.scenario = ""
        st.session_state.value_scenario = ""
        st.session_state.interview_topic = ""
        st.session_state.user_response_voluntarity = "Yes"
        st.session_state.user_age = ""
        st.session_state.user_gender = ""
        st.session_state.user_nation = ""
        st.session_state.audio_processed = False # Ensure this is reset too
        st.session_state.audio_input_key_counter = 0 # Ensure this is reset too
        navigate_to('persona_input') # Go back to the initial input screen

# --- Main App Logic (Screen Rendering) ---
if st.session_state.current_screen == 'persona_input':
    persona_input_screen()
elif st.session_state.current_screen == 'persona_display':
    persona_display_screen()
elif st.session_state.current_screen == 'interview':
    interview_screen()
elif st.session_state.current_screen == 'evaluation':
    evaluation_screen()


Overwriting app.py


In [3]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
added 22 packages in 2s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇

In [27]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.86.150.241
⠙your url is: https://curly-paths-cut.loca.lt
